## MLFlow Tracking with Python
> - Tested on macOS Monterey version 12.1 Macbook Pro, 2.2 GHz Quad-Core Intel Core i7, Memory 16GB DDR3

## Kuzushiji Character Recognition & Classification

### Background
> "Kuzushiji is a general term (including hentaigana and cursive kanji) for characters that are not used today." 

> From the Kaggle site, it is stated that Japan has millions of books and over a billion historical documents such as personal letters or diaries preserved nationwide. Most of them cannot be read by the majority of Japanese people living today because they were written in “Kuzushiji”.

> Even though Kuzushiji, a cursive writing style, had been used in Japan for over a thousand years, there are very few fluent readers of Kuzushiji today (only 0.01% of modern Japanese natives). Due to the lack of available human resources, there has been a great deal of interest in using Machine Learning to automatically recognize these historical texts and transcribe them into modern Japanese characters.

> In this notebook, the author built a deep learning model to transcribe ancient japanese letters into contemporary Japanese character

### Notebook Code 

> - Built hand-written characters recognition system using convolutional neural networks
> - Used MLflow for tracking experimetns 

![Fig1-1: Kuzushiji Characters](http://static.mxbi.net/umgy001-010-smallannomasked.jpg)

### References
> - DHQ: Ditial Humanities Quarterly: The Kuzushiji Project
> - Kuzushiji-MNIST dataset on Kaggle

In [1]:
import os
import sys
import shutil

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
import torch
import torch.nn.functional as F
import torch.nn            as nn
import torch.optim         as optim
from torch.utils.data import DataLoader
from torchvision      import datasets, transforms
from torch.autograd   import Variable

import mlflow              as mf
import mlflow.pytorch

### Hyper Parameters

In [3]:
args = {'batch_size': 4,       
        'test_batch_size': 4,       
        'epochs': 45,       
        'seed': 32,      
        'learning_rate': 0.001,    
        'momentum': 0.5}     

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Device is now set to {}'.format(device))

Device is cpu


### Awesome Convolutional Neural network 

In [4]:
class Awesome_Net(nn.Module):
    def __init__(self):
        super(Awesome_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10,  kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

### Data and Dataloader Preparation

In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.KMNIST('../data', train=True,  download=True, transform=transform)
test_dataset = datasets.KMNIST('../data', train=False, download=True, transform=transform)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True, num_workers=1)
test_loader = DataLoader(test_dataset,  batch_size=args['test_batch_size'], shuffle=True, num_workers=1)


### Model and Optimizer

In [7]:
model = Awesome_Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['learning_rate'], momentum=args['momentum'])

### Training and Testing functions

In [8]:
def train(epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 1000  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
            step = epoch * len(train_loader) + batch_idx
            mf.log_metric('train_loss',  loss.data.item())

            
def test(epoch):
    model.eval()
    test_loss = 0
    correct   = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').data.item() 
            pred = output.data.max(1)[1] 
            correct += pred.eq(target.data).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    
    print('\nAverage loss for Test dataset: {:.2f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))
        
    mf.log_metric('Test loss',     test_loss)
    mf.log_metric('Test Accuracy', test_accuracy)

In [9]:
model.train()

Awesome_Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

### MLflow Experiment

In [10]:
mf.set_tracking_uri('http://127.0.0.1:5000')

In [11]:
experiment_id = mf.set_experiment("Kuzushiji test")

In [12]:
torch.manual_seed(args["seed"])

In [13]:
with mf.start_run():
    
    mf_output_dir   = 'tmp/'
    os.makedirs(mf_output_dir, exist_ok=True)
    mf.log_params(args)

    for epoch in range(1, args['epochs'] + 1):
        train(epoch)
        test(epoch)
    
    mf.pytorch.log_model(model, "model")
    mf.log_artifacts(mf_output_dir)
    shutil.rmtree(mf_output_dir)

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.568857
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.293742
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.192515
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.825385
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.034836
Train Epoch: 1 [20000/60000 (33%)]	Loss: 1.175959
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.118869
Train Epoch: 1 [28000/60000 (47%)]	Loss: 0.549126
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.517562
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.249402
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.430155
Train Epoch: 1 [44000/60000 (73%)]	Loss: 0.212303
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.334157
Train Epoch: 1 [52000/60000 (87%)]	Loss: 0.692821
Train Epoch: 1 [56000/60000 (93%)]	Loss: 1.370765

Test set: Average loss: 0.6803, Accuracy: 5304/10000 (53%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.779669
Train Epoch: 2 [4000/60000 (7%)]	Loss: 0.220283
Train Epoch: 2 [8000/60000 (13%)]	Loss: 0.475679
Train Epoch: 2 [12000/60000 (20%)]	Loss: 1.288028
Train

Train Epoch: 11 [16000/60000 (27%)]	Loss: 0.115914
Train Epoch: 11 [20000/60000 (33%)]	Loss: 0.505866
Train Epoch: 11 [24000/60000 (40%)]	Loss: 0.596445
Train Epoch: 11 [28000/60000 (47%)]	Loss: 0.077973
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.388428
Train Epoch: 11 [36000/60000 (60%)]	Loss: 0.357540
Train Epoch: 11 [40000/60000 (67%)]	Loss: 0.386452
Train Epoch: 11 [44000/60000 (73%)]	Loss: 0.363858
Train Epoch: 11 [48000/60000 (80%)]	Loss: 0.018599
Train Epoch: 11 [52000/60000 (87%)]	Loss: 0.480685
Train Epoch: 11 [56000/60000 (93%)]	Loss: 0.514159

Test set: Average loss: 0.4216, Accuracy: 6398/10000 (64%)

Train Epoch: 12 [0/60000 (0%)]	Loss: 0.564902
Train Epoch: 12 [4000/60000 (7%)]	Loss: 1.710854
Train Epoch: 12 [8000/60000 (13%)]	Loss: 0.198724
Train Epoch: 12 [12000/60000 (20%)]	Loss: 0.032335
Train Epoch: 12 [16000/60000 (27%)]	Loss: 0.877486
Train Epoch: 12 [20000/60000 (33%)]	Loss: 0.115946
Train Epoch: 12 [24000/60000 (40%)]	Loss: 0.157156
Train Epoch: 12 [28000/60000 

Train Epoch: 21 [20000/60000 (33%)]	Loss: 0.370135
Train Epoch: 21 [24000/60000 (40%)]	Loss: 0.001011
Train Epoch: 21 [28000/60000 (47%)]	Loss: 0.454668
Train Epoch: 21 [32000/60000 (53%)]	Loss: 0.408275
Train Epoch: 21 [36000/60000 (60%)]	Loss: 0.429071
Train Epoch: 21 [40000/60000 (67%)]	Loss: 0.143282
Train Epoch: 21 [44000/60000 (73%)]	Loss: 0.497862
Train Epoch: 21 [48000/60000 (80%)]	Loss: 0.154581
Train Epoch: 21 [52000/60000 (87%)]	Loss: 0.694122
Train Epoch: 21 [56000/60000 (93%)]	Loss: 0.361934

Test set: Average loss: 0.3824, Accuracy: 6672/10000 (67%)

Train Epoch: 22 [0/60000 (0%)]	Loss: 0.359860
Train Epoch: 22 [4000/60000 (7%)]	Loss: 0.188325
Train Epoch: 22 [8000/60000 (13%)]	Loss: 0.449725
Train Epoch: 22 [12000/60000 (20%)]	Loss: 0.357778
Train Epoch: 22 [16000/60000 (27%)]	Loss: 0.063557
Train Epoch: 22 [20000/60000 (33%)]	Loss: 0.346832
Train Epoch: 22 [24000/60000 (40%)]	Loss: 0.395594
Train Epoch: 22 [28000/60000 (47%)]	Loss: 0.643486
Train Epoch: 22 [32000/60000 

Train Epoch: 31 [24000/60000 (40%)]	Loss: 0.541370
Train Epoch: 31 [28000/60000 (47%)]	Loss: 0.102786
Train Epoch: 31 [32000/60000 (53%)]	Loss: 1.977083
Train Epoch: 31 [36000/60000 (60%)]	Loss: 0.024722
Train Epoch: 31 [40000/60000 (67%)]	Loss: 0.052565
Train Epoch: 31 [44000/60000 (73%)]	Loss: 0.348886
Train Epoch: 31 [48000/60000 (80%)]	Loss: 0.141830
Train Epoch: 31 [52000/60000 (87%)]	Loss: 0.391017
Train Epoch: 31 [56000/60000 (93%)]	Loss: 0.021912

Test set: Average loss: 0.3594, Accuracy: 6791/10000 (68%)

Train Epoch: 32 [0/60000 (0%)]	Loss: 0.082276
Train Epoch: 32 [4000/60000 (7%)]	Loss: 0.871026
Train Epoch: 32 [8000/60000 (13%)]	Loss: 1.255511
Train Epoch: 32 [12000/60000 (20%)]	Loss: 0.052091
Train Epoch: 32 [16000/60000 (27%)]	Loss: 1.308279
Train Epoch: 32 [20000/60000 (33%)]	Loss: 0.488711
Train Epoch: 32 [24000/60000 (40%)]	Loss: 0.352834
Train Epoch: 32 [28000/60000 (47%)]	Loss: 0.383646
Train Epoch: 32 [32000/60000 (53%)]	Loss: 0.397713
Train Epoch: 32 [36000/60000 

Train Epoch: 41 [28000/60000 (47%)]	Loss: 0.353120
Train Epoch: 41 [32000/60000 (53%)]	Loss: 0.988855
Train Epoch: 41 [36000/60000 (60%)]	Loss: 0.006776
Train Epoch: 41 [40000/60000 (67%)]	Loss: 0.440451
Train Epoch: 41 [44000/60000 (73%)]	Loss: 0.512027
Train Epoch: 41 [48000/60000 (80%)]	Loss: 0.073973
Train Epoch: 41 [52000/60000 (87%)]	Loss: 0.010460
Train Epoch: 41 [56000/60000 (93%)]	Loss: 0.443878

Test set: Average loss: 0.3528, Accuracy: 6823/10000 (68%)

Train Epoch: 42 [0/60000 (0%)]	Loss: 0.083686
Train Epoch: 42 [4000/60000 (7%)]	Loss: 0.016458
Train Epoch: 42 [8000/60000 (13%)]	Loss: 0.365137
Train Epoch: 42 [12000/60000 (20%)]	Loss: 0.920597
Train Epoch: 42 [16000/60000 (27%)]	Loss: 0.020854
Train Epoch: 42 [20000/60000 (33%)]	Loss: 0.364359
Train Epoch: 42 [24000/60000 (40%)]	Loss: 0.537086
Train Epoch: 42 [28000/60000 (47%)]	Loss: 0.065134
Train Epoch: 42 [32000/60000 (53%)]	Loss: 0.002215
Train Epoch: 42 [36000/60000 (60%)]	Loss: 0.564088
Train Epoch: 42 [40000/60000 